In [1]:
import pandas as pd
import yaml

In [40]:
simYears = 30
thisYear = 2021

## Plan

### Here are the steps you need to complete to make this function:
-  [x] open the .yaml file with tree properties
    - [x] use these tree properties to assign a death year for each tree based on (1) species and (2) age/year planted
    
- [x] develop the architecture of the .yaml file with strategies. What will this look like? What information will it hold? How flexible/rigid will it be (at first)?

- [x] first off, based on the initial data, determine the death year of all of the trees. append this to a column 'death year' in the new dataframe

- [x]  start by making a function that simply creates the new event dataframe assuming no strategies are implemented. I.e. all the rows will have status of either "plant" or "replant." End year is determined by the death year, and replant is simply determined by the death year + 1.

- [ ]  add to that function the ability to add pruning, so the status will now be "plant," "replant," or "prune"
    * pruning will have to be implemented with some sort of year crieteria (in what year, or even better, at what age, should farmers prune?)
    * with the new row that represents the pruning event, death year carries over but is altered slightly (because we are working under the assumption that pruning increases production life)
    * unless another event is called, end year is death year
    
-  [ ] add to that function the ability to add intercropping.
    * intercropping will be implemented with (1) a year criteria and (2) a proportion criteria (what proportion of the trees will be replanted). 
    * if the trees in the originaal dataset reach the year or age criteria, the original row will become inactive because the 'end year' has been reached. The row will then be split into two rows (but both still show the same ID number, which is how we identify them as the same plot). The first row will be the trees that were left alone, and so it will be the exact same except that is has proportionally less cuerdas (the death year will carry over from the original). The second row will have the trees which were replanted. Their


In [12]:
def openYaml(yamlFilePath : str) -> dict: 
    yamlFile = open(yamlFilePath)
    parsed = yaml.load(yamlFile, Loader =yaml.FullLoader)
    return(parsed)

In [13]:
# open the yaml files  to assign attirbutes to the sim
treeAttributes = openYaml("data/trees.yml")
strategyAttributes = openYaml("intervention/strategy.yml")

In [14]:
treeAttributes

{'borbon': {'treeType': 'borbon',
  'altOrth': 'borbón',
  'cuerdaHarvestCap': 200,
  'firstHarvest': {'year': 4, 'proportion': 0.2},
  'fullHarvest': {'year': 5, 'proportion': 1.0},
  'descentHarvest': {'year': 28, 'proportionDescent': 0.2},
  'death': {'year': 30}},
 'catuai': {'treeType': 'catuai',
  'altOrth': 'catuaí',
  'cuerdaHarvestCap': 125,
  'firstHarvest': {'year': 3, 'proportion': 0.2},
  'fullHarvest': {'year': 4, 'proportion': 1.0},
  'descentHarvest': {'year': 15, 'proportionDescent': 0.2},
  'death': {'year': 17}},
 'e14': {'treeType': 'e14',
  'altOrth': 'E14',
  'cuerdaHarvestCap': 125,
  'firstHarvest': {'year': 4, 'proportion': 0.2},
  'fullHarvest': {'year': 5, 'proportion': 1.0},
  'descentHarvest': {'year': 13, 'proportionDescent': 0.2},
  'death': {'year': 15}},
 'catura': {'treeType': 'catura',
  'altOrth': 'caturra',
  'cuerdaHarvestCap': 125,
  'firstHarvest': {'year': 3, 'proportion': 0.2},
  'fullHarvest': {'year': 4, 'proportion': 1.0},
  'descentHarvest'

In [15]:
strategyAttributes

{'replant': {'isReplant': True, 'year': 1},
 'prune': {'isPrune': True, 'age': 8, 'proportion': 1.0, 'lifeExtend': 0.15},
 'intercrop': {'isIntercrop': False, 'age': 8, 'year': 8, 'proportion': 0.1}}

In [17]:
# initialData = pd.read_csv("data/demoData.csv")
upData = pd.read_csv("data/fakeData.csv")
upData.head()

,plotID,farmerName,treeType,numCuerdas,yearPlanted,ageOfTrees,productionProportion
0,0,Emilio Manzanares Garibay,catuai,7,2018,2,0.970648
1,1,Hernán Jaramillo Camacho,borbon,5,2012,8,0.720967
2,2,Angélica Horacio Jaimes,catuai,10,2015,5,0.983670
3,3,Emilio Manzanares Garibay,catuai,9,2019,1,0.821182
4,4,Pascual Rafaél Villareal Botello,borbon,6,2007,13,0.995306


In [18]:
# yearPlanted = 2020 - initialData['ageOfTrees']
# initialData['yearPlanted'] = yearPlanted
# print(initialData.keys())
# new = initialData.rename(columns={'Unnamed: 0': 'plotID'})
# new.to_csv("data/demoDataUpdated.csv",  index = False)

In [19]:
def isolateAttributes(attributes:dict, treeType:str):
    """
    Takes in a dictionary containing all of the tree attributes, as well as the name of
    the tree type, then returns a dictionary with only the attributes of the tree
    `treeType`.
    """
    
    keys = list(treeAttributes.keys())
    altOrth = [treeAttributes[key]['altOrth'] for key in treeAttributes]
    # tipos = keys + altOrth # all of the possible spellings for the tree types
            
    if treeType in keys:
        treeDict = treeAttributes[treeType]
                
    elif treeType in altOrth:
        keyPair = [(key, treeAttributes[key]['altOrth']) for key in treeAttributes]
        _treeType = ''
        for i,e in enumerate(keyPair):
            if treeType == e[1]: # if it's the altOrth
                _treeType = e[0] # key to the key
                
            
            if len(_treeType) > 0:
                treeDict = treeAttributes[_treeType]
                
            else:
                raise AttributeError(
                """
                '%s' is not a recognized value (orthography) in the `treeAttributes` dict.
                
                """%(treeType))
                
    else:
        raise AttributeError(
        """
        '%s' is not a recognized value (orthography) in the `treeAttributes` dict.
                
        """%(treeType))
        
    return(treeDict)

In [32]:
def transformData(year:int,
                  simulationYears:int,
                  farmData:pd.DataFrame,
                  treeAttributes:dict=None,
                  strategyAttributes:dict=None):
    """
    takes in data from repository and returns a new,  transformed dataframe that
    tracks events.
    
    year is an int of the  year where the simulation starts.  if the simulation moves forward
    from  the present, the year is the current year. else it is  the  year the simulation
    begins.
    
    simulationYears is the amount of years that the simulation will iterate through. This
    is necessary to make sure the transformed data only captures events within this range. 
    
    farm data is data frame with farmer's plots
    
    tree attributes is dictionary opened from yaml file with attritbutes of trees.
    
    strategy attributes is dictionary opened from yaml file with attributes of strategies.
    
    returns dataframe with events
    
    
    Notes
    -----
    
    as of now, the condition is that intercrop year and prune year are not in the same. but I might be able to figure out how to work that out. 
    """
    
    endYear = year + simulationYears
    
    # iterate through each row of the original plot dataframe
    for i in range(len(farmData)):
        plotID = farmData["plotID"][i]
        farmerName = farmData["farmerName"][i]
        treeType = farmData["treeType"][i]
        numCuerdas =  farmData["numCuerdas"][i]
        startYear = farmData["yearPlanted"][i]
        productionProportion = farmData["productionProportion"][i]
        
        # assume all are planted for initialization
        status = "plant" 

        treeAge = year - startYear

        # check to see that this tree exists in config file
        # _altOrth = [treeAttributes[item]["altOrth"] for item in treeAttributes]
        
        if treeAttributes:
            # isolate the dictionary we are concerned with on this plot
            treeDict = isolateAttributes(attributes=treeAttributes, treeType=treeType)

            # isolate individual variables from this dict
            cuerdaHarvestCap  = treeDict["cuerdaHarvestCap"]
            firstHarvest = treeDict["firstHarvest"]
            fullHarvest = treeDict["fullHarvest"]
            descentHarvest = treeDict["descentHarvest"]
            death = treeDict["death"]

            # calculate death year
            yearsTillDeath = death["year"] - treeAge
            deathYear = year + yearsTillDeath

            # create the initial row for the transformed dataframe
            row = pd.DataFrame([[plotID, farmerName, treeType, numCuerdas, status, 
                                      startYear, deathYear, productionProportion]], columns=["plotID", "farmerName", "treeType", 
                                                "numCuerdas", "status", "startYear",
                                               "deathYear", "productionProportion"])
           
            # if this is the first row of the whole transformation
            if (i == 0):
                # initialize the transformation dataframe
                transformedData = row

            else:
                transformedData = pd.concat([transformedData, row], ignore_index=True)
                #transformedData  = transformedData.reset_index(drop = True, inplace = True)
                
                
            # now you've transformed all of the original entries to the new format
            # now you should be iterating through transformed data to add events

            # check to see if replant is in strategy (it always should be)
            if strategyAttributes["replant"]["isReplant"] ==  True:
                replantYear = (deathYear + 1)
            else:
                replantYear = None

            # check to see if prune  is in strategy config
            if strategyAttributes["prune"]["isPrune"] ==  True:
                pruneAge = strategyAttributes["prune"]["age"]
                lifeExtend = strategyAttributes["prune"]["lifeExtend"]
            else:
                pruneAge = None

            # check to see if  intercrop is in strategy config
            if strategyAttributes["intercrop"] == True:
                intercropAge = strategyAttributes["prune"]["age"]
            else:
                intercropAge = None

            # for this specific plot (see plotID),
            # create a row to check against to see if the program needs to continue creating events
            checkRow = row

            # create a  new var for the year of transformation for this plot
            simYear = year
            # create a new var for the tree's age for this plot
            simTreeAge = treeAge

            # iterate through all years of the simulation to check event sequences
            while (simYear < endYear):
               #  isolate dict
                deathYear = checkRow["deathYear"][0]


                if (replantYear):
                    # death takes precedence over pruning 
                    if (simYear == deathYear):
                        # update death year
                        simTreeAge = -1
                        deathYear = (simYear +  1) + death["year"]
                        status = "replant"
                        nextRow = pd.DataFrame([[plotID, farmerName, treeType, numCuerdas, status, replantYear, deathYear,productionProportion]], 
                                            columns=["plotID", "farmerName","treeType", "numCuerdas", "status", "startYear","deathYear","productionProportion"])
                        transformedData  = pd.concat([transformedData, nextRow],  ignore_index=True)
                        checkRow = nextRow
                        simYear += 1
                        simTreeAge += 1
                        
                        # no more than one action per year IF action is death
                        continue


                    elif (pruneAge):
                        if (simTreeAge == pruneAge):
                            # add years proportional to tree's lifespan:
                            addedYears = round((death["year"] * lifeExtend))
                            adjustedDeathYear = (checkRow["deathYear"][0]) + addedYears
                            pruneYear = simYear
                            status = "prune"
                            nextRow = pd.DataFrame([[plotID, farmerName,treeType, numCuerdas, status, pruneYear, adjustedDeathYear, productionProportion]], 
                                                   columns=["plotID", "farmerName","treeType", "numCuerdas", "status", "startYear","deathYear","productionProportion"])
                            transformedData  = pd.concat([transformedData, nextRow],  ignore_index=True)
                            checkRow  = nextRow
                            simYear += 1
                            simTreeAge += 1
                            continue

                        else:
                            simYear += 1
                            simTreeAge += 1
                            continue
                            
                    else:
                        simYear += 1
                        simTreeAge += 1
                        continue

                else:
                    simYear += 1
                    simTreeAge += 1
                    continue



        else:
            print("No tree attributes!!!")
            print(treeType)
            break
            
        
    return(transformedData)

Right now productionProportion column is not functioning in relationship to events

In [38]:
simData = transformData(year=startYear, simulationYears=simYears, farmData=upData, treeAttributes=treeAttributes, strategyAttributes=strategyAttributes)
simData[30:80]

,plotID,farmerName,treeType,numCuerdas,status,startYear,deathYear,productionProportion
30,8,Timoteo Quiñónez Miramontes,catuai,6,prune,2024,2036,0.716944
31,8,Timoteo Quiñónez Miramontes,catuai,6,replant,2034,2054,0.716944
32,8,Timoteo Quiñónez Miramontes,catuai,6,prune,2045,2057,0.716944
33,9,Pascual Rafaél Villareal Botello,borbon,9,plant,1992,2022,0.849839
34,9,Pascual Rafaél Villareal Botello,borbon,9,replant,2023,2053,0.849839
35,9,Pascual Rafaél Villareal Botello,borbon,9,prune,2031,2057,0.849839
36,10,Jerónimo Aponte,catuai,7,plant,2019,2036,0.829217
37,10,Jerónimo Aponte,catuai,7,prune,2027,2039,0.829217
38,10,Jerónimo Aponte,catuai,7,replant,2037,2057,0.829217
39,10,Jerónimo Aponte,catuai,7,prune,2048,2060,0.829217


# To do:

- [ ] review farm class and see what's going on
- [ ] figure out how to incorporate classes into what is happening here

In [47]:
# group by the individual ids for iteration
byID = simData.groupby("plotID")

# iterate through each plot
for plotID, group in byID:
    #print(plotID)
    
    # iterate through each event for each plot
    for eventNumber, event in group.iterrows():
        #print(event["farmerName"])
        x = 0
        

In [36]:
def simulate(startYear:int, simultationYears:int, eventData:pd.DataFrame):
    
    for i in range(simultationYears):
        simYear = startYear + i
        
        
        

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [51]:
round?

Signature: round(number, ndigits=None)
Docstring:
Round a number to a given precision in decimal digits.

The return value is an integer if ndigits is omitted or None.  Otherwise
the return value has the same type as the number.  ndigits may be negative.
Type:      builtin_function_or_method
